In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as du
from torch.utils.data import IterableDataset

import numpy as np
import sys
import csv
from os import listdir
from tqdm import tqdm

In [2]:
class Price_Dataset(IterableDataset):
	def __init__(self, directory, window_size = 100, start = 0, end = 8):
		super().__init__()

		self.window_size = window_size

		prices = []
		files = listdir(directory)
		csvs = [f'{directory}/{f}' for f in files if f.endswith('csv')][start:end]
		for c in csvs:
			with open(c, 'r') as f:
				lines = list(csv.reader(f))[1:]
				prices.append(np.array([float(l[5]) for l in lines]))
		prices = np.array(prices)
		for i, row in enumerate(prices):
			for j, col in enumerate(row[:-1]):
				prices[i][j] -= prices[i][j+1]

		self.prices = prices
		self.length = int(len(self.prices)*(
			(len(self.prices[0]) - self.window_size-1)+(len(self.prices[0]) - self.window_size-1)//2+
			(len(self.prices[0]) - self.window_size-1)//3+(len(self.prices[0]) - self.window_size-1)//4+
			(len(self.prices[0]) - self.window_size-1)//5))

	def __len__(self):
		return self.length

	def __iter__(self):
		for p in self.prices:
			for k in np.arange(1, 5):
				for offset in range(0, (len(p) - self.window_size-1)//k):
					out = torch.zeros(self.window_size)
					for i in range(self.window_size):
						out[i] = np.sum(p[offset+i*k:offset+(i+1)*k])
					out = out.unsqueeze(1)
					label = np.sum(p[offset+self.window_size*k:offset+(self.window_size+1)*k])
					yield out, label

In [13]:
class Transformer_Model(nn.Module):
    def __init__(self, d_model, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.trans = nn.Transformer(d_model=d_model)
        # self.fc1 = nn.Linear() 
    def forward(self, x):
        x = self.trans(x)
        return x

In [15]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")
epochs = 5
learning_rate = 1e-4
batch_size = 20
window_size = 80
# start = 0
# end = 4

train_dataset = Price_Dataset('Kline/filter', window_size=window_size)
train_loader = du.DataLoader(dataset = train_dataset, batch_size = batch_size)

net = Transformer_Model(window_size, 1, 1, 1)
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

loss_func = nn.MSELoss()
a,b = next(iter(train_loader))
# net = 3

using device: cuda:0


In [16]:

for epoch in range(1 , epochs + 1):
    sum_loss = 0
    for batch_idx, (price, label) in enumerate(tqdm(train_loader)):
        price, label = price.to(device), label.unsqueeze(1).to(device)

        optimizer.zero_grad()

        pred = net(price)
        print(pred.shape)

    #     loss = loss_func(pred.float(), label.float())
    #     loss.backward()

    #     optimizer.step()

    #     sum_loss += loss.item()

    # print(f'Epoch {epoch}: Train loss: {sum_loss/batch_idx/batch_size:.8f}')

  0%|          | 0/839 [00:00<?, ?it/s]


TypeError: forward() missing 1 required positional argument: 'tgt'